In [6]:
!pip install pandas numpy scikit-learn matplotlib seaborn

     ---------------------------------------- 0.0/162.8 kB ? eta -:--:--
     ------- ------------------------------- 30.7/162.8 kB 1.3 MB/s eta 0:00:01
     -------------------------- ----------- 112.6/162.8 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 162.8/162.8 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.7 MB ? eta -:--:--
    --------------------------------------- 0.1/7.7 MB 6.8 MB/s eta 0:00:02
   - -------------------------------------- 0.2/7.7 MB 3.8 MB/s eta 0:00:02
   - -------------------------------------- 0.3/7.7 MB 3.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.5/7.7 MB 2.6 MB/s eta 0:00:03
   --- ------------------------------------ 0.6/7.7 MB 3.1 MB/s eta 0:00:03
   ---- ----------------------------------- 0.8/7.7 MB 3.1 MB/s eta 0:00:03
   -------- ------------------------------- 1.6/7.7 MB 5.0 MB/s eta 0:00:02
   -------- ------------------------------- 1.6/7.7 MB 5.0 MB/s eta 0:00:02
   -------

In [2]:
import pandas as pd
import numpy as np

In [26]:
# Load the data
boxes_df = pd.read_csv('boxes.csv')
purchase_df = pd.read_csv('purchase.csv')
problem_df = pd.read_csv('problem 1.csv')

In [30]:
# Convert the purchase date to datetime format
purchase_df['PURCHASE_DATE'] = pd.to_datetime(purchase_df['PURCHASE_DATE'],format='%d/%m/%Y')

# Filter purchases made in the first 15 days of March 2019
march_purchases = purchase_df[(purchase_df['PURCHASE_DATE'].dt.month == 3) & (purchase_df['PURCHASE_DATE'].dt.day <= 15)]

# Merge the purchase data with the boxes data to get details about the purchased boxes
merged_df = pd.merge(march_purchases, boxes_df, on='BOX_ID')

# Group by MAGIC_KEY and check if they purchased milk or meat
purchase_summary = merged_df.groupby('MAGIC_KEY').agg({'MILK': 'sum', 'MEAT': 'sum'})

# Predict which Magic Keys will make a purchase
# If a Magic Key purchased milk or meat, they will make a purchase
purchase_summary['PURCHASE'] = purchase_summary.apply(lambda row: 'Y' if row['MILK'] > 0 or row['MEAT'] > 0 else 'N', axis=1)


In [32]:
# Merge the problem data with our purchase summary to get the final prediction
submission_df = pd.merge(problem_df, purchase_summary[['PURCHASE']], on='MAGIC_KEY', how='left')

# Fill missing values with 'N' and ensure the column is of object dtype to avoid dtype incompatibility
submission_df['PURCHASE'] = submission_df['PURCHASE'].fillna('N').astype(object)

# Save the submission file
submission_df.to_csv('submission.csv', index=False)
